# A simple pandas example

In this notebook I'm just loading some data into a pandas dataframe and putting it through the following process:

* select only the columns I need
* cleaning it up (trimming whitespace from values)
* typing the columns (supporting multiple date formats)
* removing invalid rows (where required values are null)

There is much more that could be done, using pandas and/or adding numpy.

References:
- https://www.codecademy.com/article/introduction-to-numpy-and-pandas
- https://duckdb.org/2021/05/14/sql-on-pandas.html



## Get some simple data

In [2]:
!git clone https://github.com/prule/data.git data

Cloning into 'data'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
Receiving objects: 100% (18/18), done.
remote: Total 18 (delta 0), reused 18 (delta 0), pack-reused 0


In [3]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=fba271f3dbdcf48c07d555e78408c95da203a8f3bc546030d13ad3be1974548d
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


## Load all the transactions files into a dataframe

In [4]:
import pandas as pd
import os

# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.options.mode.copy_on_write = True

# Define the directory containing the CSV files
directory = './data/sample1/transactions'

# Load all CSV files in the directory into a list of DataFrames
csv_files = [pd.read_csv(os.path.join(directory, filename)) for filename in os.listdir(directory) if filename.endswith('.csv')]

# Concatenate the DataFrames into a single DataFrame
# https://stackoverflow.com/a/47576083/20242
df = pd.concat(csv_files,ignore_index=True)

df.head()


,date,account,description,amount,location,comment
0,2020-13-01,x,burger,0.01,unknown location,invalid date
1,invalid date,x,petrol,0.02,,invalid date
2,NaN,x,tennis,0.03,NaN,no date
3,2020-12-01,,tennis,0.04,NaN,blank (many spaces) account
4,2020-12-01,x,petrol,x,NaN,invalid number for amount


## Select only the columns needed

In [5]:
# select only the columns needed
selected = df[['date','description','amount']]
selected.head()

,date,description,amount
0,2020-13-01,burger,0.01
1,invalid date,petrol,0.02
2,NaN,tennis,0.03
3,2020-12-01,tennis,0.04
4,2020-12-01,petrol,x


## Clean data

In [21]:
cleaned = selected
## date
# trim whitespace
cleaned['date'] = cleaned['date'].str.strip()

## Type conversion

In [22]:
typed = cleaned

# amount - type conversion
typed['amount'] = pd.to_numeric(typed['amount'], errors='coerce')

# type conversion using multiple formats
typed['date_new'] = None # add column to populate using each date format
typed.loc[typed['date_new'].isnull(), 'date_new'] = pd.to_datetime(typed['date'], format='%Y-%m-%d', errors='coerce')
typed.loc[typed['date_new'].isnull(), 'date_new'] = pd.to_datetime(typed['date'], format='%d-%m-%Y', errors='coerce')
# select the right columns (aliasing)
typed = typed[['date_new','description','amount']].rename(columns={'date_new':'date'})
typed.head()

,date,description,amount
0,NaT,burger,0.01
1,NaT,petrol,0.02
2,NaT,tennis,0.03
3,2020-12-01 00:00:00,tennis,0.04
4,2020-12-01 00:00:00,petrol,NaN


In [8]:
typed.count()

date           17
description    22
amount         19
dtype: int64

## Remove invalid rows

In [9]:
# df = df.drop(df[(df.score < 50) & (df.score > 20)].index)
valid = typed.drop(typed[(typed.date.isnull()) | (typed.amount.isnull())].index)
valid.head(100)

,date,description,amount
3,2020-12-01 00:00:00,tennis,0.04
7,2020-03-01 00:00:00,burger,15.47
8,2020-03-03 00:00:00,tennis,35.03
9,2020-03-04 00:00:00,petrol,150.47
10,2020-03-05 00:00:00,petrol,50.47
11,2020-03-05 00:00:00,petrol,50.48
12,2020-02-01 00:00:00,burger,15.46
13,2020-02-02 00:00:00,movies,20.01
14,2020-02-03 00:00:00,tennis,35.01
15,2020-02-04 00:00:00,petrol,150.46


In [10]:
valid.count()

date           16
description    16
amount         16
dtype: int64

## Demonstrate how we can apply sql to further transform.

In [16]:
import pandasql as ps

rs = ps.sqldf("select * from valid order by amount desc")

rs.head()

,date,description,amount
0,2020-02-04 00:00:00.000000,electricity,300.47
1,2020-03-04 00:00:00.000000,petrol,150.47
2,2020-02-04 00:00:00.000000,petrol,150.46
3,2020-01-04 00:00:00.000000,petrol,150.45
4,2020-01-04 00:00:00.000000,petrol,150.45


## Demonstrate duckdb as an alternative to pandasql

In [14]:
!pip install duckdb

In [17]:
import duckdb

rs = duckdb.query("select * from valid order by amount desc").df()

rs.head()

,date,description,amount
0,2020-02-04,electricity,300.47
1,2020-03-04,petrol,150.47
2,2020-02-04,petrol,150.46
3,2020-01-04,petrol,150.45
4,2020-01-04,petrol,150.45


In [20]:
rs = duckdb.query("select * from valid where amount < 20 order by amount desc").df()

rs.head()

,date,description,amount
0,2020-03-01,burger,15.47
1,2020-02-01,burger,15.46
2,2020-01-01,burgers,15.45
3,2020-12-01,tennis,0.04
